# Structuring ECOICOP v1 information for classification

The ECOICOP v1 classification contains unstructured information for its most detailed level of classification. This notebook leverages LLMs in order to structure this information in a format which can be used for similarity search.

In [ ]:
import os
import time # Add delays because of free API rate limits
from datetime import datetime

import pandas as pd

from typing import List
from pydantic import BaseModel
from pydantic_ai import Agent
from pydantic_ai.models.mistral import MistralModel
from pydantic_ai.models.groq import GroqModel

from dotenv import dotenv_values

In [ ]:
import nest_asyncio # Fix issues with Jupyter notebook event loop
nest_asyncio.apply()

Import environment variables with API keys

In [ ]:
config = dotenv_values(".env")

Define the Pydantic model for parsing the additional information in each ECOICOP level 4

In [ ]:
class EcoicopDetails(BaseModel):
    examples: List[str]

Read ECOICOP definitions (only selected columns)

In [ ]:
usecols = ["CODE", "NAME_EN", "LEVEL", "PARENT_CODE", "InclusionNote"]

data_df = pd.read_csv(
    filepath_or_buffer="ecoicop_v1/ECOICOP-HICP_Structure_Labels.tsv", 
    sep="\t",
    usecols=usecols,)

Filter only level 4 items

In [ ]:
data_df = data_df.loc[data_df["LEVEL"] == 4]

In [ ]:
data_dict = data_df.to_dict(orient="records")

## Mistral

Initialize PydanticAI agent to structure information

In [ ]:
llm_model = "mistral-large-latest"

In [ ]:
model = MistralModel(model_name=llm_model, api_key=config.get("MISTRAL_API_KEY"))
agent = Agent(
    model=model,
    retries=3,
    result_type=EcoicopDetails,
    system_prompt=(
        'You are an expert data curator. You will receive a string of text '
        'with examples of items to be included into a certain classification. '
        'Your task is to transform this text into a list of self-explainig '
        'items descriptions, exploding the examples to the most granular level you can '
        'identify in the text. '
        'Each item description should contain all necessary information for classification '
        'as provided in the original text. Prefer complete descriptions rather than single words. '
        'If possible, avoid the use of ambiguous or generic terms such as `other` or `miscellaneous`. '
        ),
)

Run calls to the Agent to extract and format information

In [ ]:
results = []
failed_calls = []
for i, item in enumerate(data_dict):
    # Print every 20 items to show progress
    if i % 20 == 0:
        print(f"Processing item {i+1} out of {len(data_dict)}")
    # Add switch to skip none items
    if item.get("InclusionNote") is None:
        # No information to parse, just append existing item
        results.append(item)
        continue
    # Time delay to respect API rate limits
    time.sleep(3)
    try:
        prompt = "{}. {}".format(item.get("NAME_EN"), item.get("InclusionNote"))
        agent_result = agent.run_sync(prompt, model_settings={'temperature': 0.0})
        # Add to results all examples, including the original class name...list concatenation
        for ex in [item.get("NAME_EN")] + agent_result.data.model_dump().get("examples"):
            # the new "NAME_EN" is inserted at the end, so it overwrites the original one
            results.append({**item, "NAME_EN": ex})
    except Exception as e:
        failed_calls.append(item)
        print(f"Error processing item {i+1} out of {len(data_dict)}")
        print(item)
        print(e)
        continue

Save results and failed calls

In [ ]:
results_df = pd.DataFrame(results)
results_df.drop(columns=["InclusionNote", "LEVEL"]).to_csv(
    "results/ecoicop_v1_{}_{}.csv".format(
        llm_model,
        datetime.now().strftime("%Y-%m-%d_%H%M%S")),
    index=False)

In [ ]:
failed_df = pd.DataFrame(failed_calls)
failed_df.to_csv(
    "results/failed_ecoicop_v1_{}_{}.csv".format(
        llm_model,
        datetime.now().strftime("%Y-%m-%d_%H%M%S")),
    index=False)

## Llama 3.3 on Groq

In [ ]:
llm_model = "llama-3.3-70b-versatile"

In [ ]:
model = GroqModel(
    model_name=llm_model, 
    api_key=config.get("GROQ_API_KEY"))
agent = Agent(
    model=model,
    retries=3,
    result_type=EcoicopDetails,
    system_prompt=(
        'You are an expert data curator. You will receive a string of text '
        'with examples of items to be included into a certain classification. '
        'Your task is to transform this text into a list of self-explainig '
        'items descriptions, exploding the examples to the most granular level you can '
        'identify in the text. '
        'Each item description should contain all necessary information for classification '
        'as provided in the original text. Prefer complete descriptions rather than single words. '
        'If possible, avoid the use of ambiguous or generic terms such as `other` or `miscellaneous`. '
        ),
)

In [ ]:
results = []
failed_calls = []
for i, item in enumerate(data_dict):
    # Print every 20 items to show progress
    if i % 20 == 0:
        print(f"Processing item {i+1} out of {len(data_dict)}")
    # Add switch to skip none items
    if item.get("InclusionNote") is None:
        # No information to parse, just append existing item
        results.append(item)
        continue
    # Time delay to respect API rate limits
    time.sleep(3)
    try:
        prompt = "{}. {}".format(item.get("NAME_EN"), item.get("InclusionNote"))
        agent_result = agent.run_sync(prompt, model_settings={'temperature': 0.0})
        # Add to results all examples, including the original class name...list concatenation
        for ex in [item.get("NAME_EN")] + agent_result.data.model_dump().get("examples"):
            # the new "NAME_EN" is inserted at the end, so it overwrites the original one
            results.append({**item, "NAME_EN": ex})
    except Exception as e:
        failed_calls.append(item)
        print(f"Error processing item {i+1} out of {len(data_dict)}")
        print(item)
        print(e)
        continue

In [ ]:
results_df = pd.DataFrame(results)
results_df.drop(columns=["InclusionNote", "LEVEL"]).to_csv(
    "results/ecoicop_v1_{}_{}.csv".format(
        llm_model,
        datetime.now().strftime("%Y-%m-%d_%H%M%S")),
    index=False)

In [ ]:
failed_df = pd.DataFrame(failed_calls)
failed_df.to_csv(
    "results/failed_ecoicop_v1_{}_{}.csv".format(
        llm_model,
        datetime.now().strftime("%Y-%m-%d_%H%M%S")),
    index=False)

## Deepseek on Groq

In [ ]:
llm_model = "deepseek-r1-distill-llama-70b"

In [ ]:
model = GroqModel(
    model_name=llm_model, 
    api_key=config.get("GROQ_API_KEY"))
agent = Agent(
    model=model,
    retries=3,
    result_type=EcoicopDetails,
    system_prompt=(
        'You are an expert data curator. You will receive a string of text '
        'with examples of items to be included into a certain classification. '
        'Your task is to transform this text into a list of self-explainig '
        'items descriptions, exploding the examples to the most granular level you can '
        'identify in the text. '
        'Each item description should contain all necessary information for classification '
        'as provided in the original text. Prefer complete descriptions rather than single words. '
        'If possible, avoid the use of ambiguous or generic terms such as `other` or `miscellaneous`. '
        ),
)

In [ ]:
results = []
failed_calls = []
for i, item in enumerate(data_dict):
    # Print every 20 items to show progress
    if i % 20 == 0:
        print(f"Processing item {i+1} out of {len(data_dict)}")
    # Add switch to skip none items
    if item.get("InclusionNote") is None:
        # No information to parse, just append existing item
        results.append(item)
        continue
    # Time delay to respect API rate limits
    time.sleep(3)
    try:
        prompt = "{}. {}".format(item.get("NAME_EN"), item.get("InclusionNote"))
        agent_result = agent.run_sync(prompt, model_settings={'temperature': 0.0})
        # Add to results all examples, including the original class name...list concatenation
        for ex in [item.get("NAME_EN")] + agent_result.data.model_dump().get("examples"):
            # the new "NAME_EN" is inserted at the end, so it overwrites the original one
            results.append({**item, "NAME_EN": ex})
    except Exception as e:
        failed_calls.append(item)
        print(f"Error processing item {i+1} out of {len(data_dict)}")
        print(item)
        print(e)
        continue

In [ ]:
results_df = pd.DataFrame(results)
results_df.drop(columns=["InclusionNote", "LEVEL"]).to_csv(
    "results/ecoicop_v1_{}_{}.csv".format(
        llm_model,
        datetime.now().strftime("%Y-%m-%d_%H%M%S")),
    index=False)

In [ ]:
failed_df = pd.DataFrame(failed_calls)
failed_df.to_csv(
    "results/failed_ecoicop_v1_{}_{}.csv".format(
        llm_model,
        datetime.now().strftime("%Y-%m-%d_%H%M%S")),
    index=False)

# Combine results

In [ ]:
result_files = [f for f in os.listdir("results/") if f.startswith("ecoicop_v1") and f.endswith(".csv")]

In [ ]:
results_list = []
for f in result_files:
    temp_df = pd.read_csv(os.path.join("results", f))
    results_list.append(temp_df)

results_df = pd.concat(results_list)

Normalize to lowercase and remove all "other" labels

In [ ]:
# Lowercase
results_df["NAME_EN"] = results_df["NAME_EN"].str.lower()
# Remove duplicates
results_df = results_df.drop_duplicates(ignore_index=True)
# Remove items with "other" or "miscellaneous"
results_df = results_df[~results_df["NAME_EN"].str.contains("other|miscellaneous")]

In [ ]:
results_df.to_csv(
    "results/consolidated_ecoicop_v1_{}.csv".format(datetime.now().strftime("%Y-%m-%d")), 
    index=False)